**Realizando os devidos imports**

In [ ]:
import pandas as pd
from sklearn.datasets import fetch_openml
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
import numpy as np
from sklearn import datasets
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV, cross_validate, train_test_split
from sklearn.metrics import roc_curve, roc_auc_score, precision_recall_curve, f1_score, auc
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [ ]:

# Importação GridSearchCV.
from sklearn.model_selection import GridSearchCV

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving sample_submission.csv to sample_submission.csv
Saving test.csv to test.csv
Saving train.csv to train.csv


 Extracting data

In [ ]:
df_test = pd.read_csv("test.csv")
df_train = pd.read_csv("train.csv")
df = pd.read_csv("sample_submission.csv")

Data overview

In [ ]:
df.head(5)
## Example of final dataset

,PassengerId,Transported
0,0013_01,False
1,0018_01,False
2,0019_01,False
3,0021_01,False
4,0023_01,False


# Dataset used for training

In [ ]:
df_train.head(2)

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True


# Dataset with no labels that we will try to predict

In [ ]:
df_test.head(4)

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter


# Taking a look of how it is our df_train

In [ ]:
print('df_train.shape',df_train.shape)
print('df_train.columns',df_train.columns)

df_train.shape (8693, 14)
df_train.columns Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age',
       'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Name', 'Transported'],
      dtype='object')


In [ ]:
dataTypeSeries = df_train.dtypes
print('Data type of each column of Dataframe :')
print(dataTypeSeries)

Data type of each column of Dataframe :
PassengerId      object
HomePlanet       object
CryoSleep        object
Cabin            object
Destination      object
Age             float64
VIP              object
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Name             object
Transported        bool
dtype: object


In [ ]:
df_train.describe()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
count,8514.000000,8512.000000,8510.000000,8485.000000,8510.000000,8505.000000
mean,28.827930,224.687617,458.077203,173.729169,311.138778,304.854791
std,14.489021,666.717663,1611.489240,604.696458,1136.705535,1145.717189
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,38.000000,47.000000,76.000000,27.000000,59.000000,46.000000
max,79.000000,14327.000000,29813.000000,23492.000000,22408.000000,24133.000000


In [ ]:
## categorical variable
print('df_train.HomePlanet.value_counts()',df_train.HomePlanet.value_counts())
print('---------------------------------------------------------------')

## binary categorical variable
print('df_train.CryoSleep.value_counts()',df_train.CryoSleep.value_counts())
print('---------------------------------------------------------------')

## numerical discret variable
print('df_train.Cabin.value_counts()',df_train.Cabin.value_counts())
print('---------------------------------------------------------------')

## categorical variable
print('df_train.Destination.value_counts()',df_train.Destination.value_counts())
print('---------------------------------------------------------------')


## binary categorical variable
print('df_train.VIP.value_counts()',df_train.VIP.value_counts())


df_train.HomePlanet.value_counts() Earth     4602
Europa    2131
Mars      1759
Name: HomePlanet, dtype: int64
---------------------------------------------------------------
df_train.CryoSleep.value_counts() False    5439
True     3037
Name: CryoSleep, dtype: int64
---------------------------------------------------------------
df_train.Cabin.value_counts() G/734/S     8
G/109/P     7
B/201/P     7
G/1368/P    7
G/981/S     7
           ..
G/556/P     1
E/231/S     1
G/545/S     1
G/543/S     1
F/947/P     1
Name: Cabin, Length: 6560, dtype: int64
---------------------------------------------------------------
df_train.Destination.value_counts() TRAPPIST-1e      5915
55 Cancri e      1800
PSO J318.5-22     796
Name: Destination, dtype: int64
---------------------------------------------------------------
df_train.VIP.value_counts() False    8291
True      199
Name: VIP, dtype: int64


# We have 14 columns, that:
- 2 who are binary (CryoSleep, VIP)
- 6 who are  continuos quantitative (Age,RoomService,FoodCourt,ShoppingMall, Spa,VRDeck )
- 2 are categorical (HomePlanet,Destination)
- 1 is numerical discret variable (Cabin)

Other 3 are: name, id, transportaded

In [ ]:
df_train.head(1)

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False


# One hot enconding for categorical variables


In [ ]:
# Get one hot encoding of columns 
one_hot = pd.get_dummies(df_train['HomePlanet'])

# Join the encoded df
df_train = df_train.join(one_hot)


##----------------------------------
# Get one hot encoding of columns 
one_hot = pd.get_dummies(df_train['Destination'])

# Join the encoded df
df_train = df_train.join(one_hot)


##----------------------------------
print(df_train.head(1))


  PassengerId HomePlanet CryoSleep  Cabin  Destination   Age    VIP  \
0     0001_01     Europa     False  B/0/P  TRAPPIST-1e  39.0  False   

   RoomService  FoodCourt  ShoppingMall  Spa  VRDeck             Name  \
0          0.0        0.0           0.0  0.0     0.0  Maham Ofracculy   

   Transported  Earth  Europa  Mars  55 Cancri e  PSO J318.5-22  TRAPPIST-1e  
0        False      0       1     0            0              0            1  


Some columns need to be renamed to make our job easier when calling them

In [ ]:
df_train.rename(columns={'55 Cancri e':'55_Cancri_e',
                         'PSO J318.5-22':'PSO_J318_5_22','TRAPPIST-1e':'TRAPPIST_1e'},inplace=True)

# Checking nan values  in dataframe

In [ ]:
df_train.isnull().sum()

PassengerId        0
HomePlanet       201
CryoSleep        217
Cabin            199
Destination      182
Age              179
VIP              203
RoomService      181
FoodCourt        183
ShoppingMall     208
Spa              183
VRDeck           188
Name             200
Transported        0
Earth              0
Europa             0
Mars               0
55_Cancri_e        0
PSO_J318_5_22      0
TRAPPIST_1e        0
dtype: int64

In [ ]:
df_train.CryoSleep.value_counts()

False    5439
True     3037
Name: CryoSleep, dtype: int64

In [ ]:
## As there are few nan values, lets fill them with the value that appears more. In this case it is False

In [ ]:
df_train.CryoSleep.fillna('False',inplace=True)

# Fillna in Cabin

In [ ]:
 df_train.Cabin.value_counts(normalize=True)
## lets not use this variable

G/734/S     0.000942
G/109/P     0.000824
B/201/P     0.000824
G/1368/P    0.000824
G/981/S     0.000824
              ...   
G/556/P     0.000118
E/231/S     0.000118
G/545/S     0.000118
G/543/S     0.000118
F/947/P     0.000118
Name: Cabin, Length: 6560, dtype: float64

# Fillna in VIP

In [ ]:
df_train.VIP.value_counts()

False    8291
True      199
Name: VIP, dtype: int64

In [ ]:
## As there are few nan values, lets fill them with the value that appears more. In this case it is False

In [ ]:
df_train.VIP.fillna('False',inplace=True)

##  FIllna in numerical variables with their mean

In [ ]:
df_train[['Age','RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']].describe()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
count,8514.000000,8512.000000,8510.000000,8485.000000,8510.000000,8505.000000
mean,28.827930,224.687617,458.077203,173.729169,311.138778,304.854791
std,14.489021,666.717663,1611.489240,604.696458,1136.705535,1145.717189
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,38.000000,47.000000,76.000000,27.000000,59.000000,46.000000
max,79.000000,14327.000000,29813.000000,23492.000000,22408.000000,24133.000000


In [ ]:
df_train.Age.fillna(28,inplace=True)
df_train.RoomService.fillna(224,inplace=True)
df_train.FoodCourt.fillna(458,inplace=True)
df_train.ShoppingMall.fillna(173,inplace=True)
df_train.Spa.fillna(311,inplace=True)
df_train.VRDeck.fillna(304,inplace=True)

In [ ]:
df_train['new_CryoSleep']=df_train['CryoSleep'].apply(lambda x:  0 if x== 'False' else 1)

# Checking if there are any more nan values

In [ ]:
df_train.isnull().sum()

PassengerId        0
HomePlanet       201
CryoSleep          0
Cabin            199
Destination      182
Age                0
VIP                0
RoomService        0
FoodCourt          0
ShoppingMall       0
Spa                0
VRDeck             0
Name             200
Transported        0
Earth              0
Europa             0
Mars               0
55_Cancri_e        0
PSO_J318_5_22      0
TRAPPIST_1e        0
new_CryoSleep      0
dtype: int64

In [ ]:
df_model = df_train[['Earth','Europa','Mars','new_CryoSleep',
                     'TRAPPIST_1e','55_Cancri_e','PSO_J318_5_22',
                     'Age','RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']]

In [ ]:
df_model

,Earth,Europa,Mars,new_CryoSleep,TRAPPIST_1e,55_Cancri_e,PSO_J318_5_22,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
0,0,1,0,1,1,0,0,39.0,0.0,0.0,0.0,0.0,0.0
1,1,0,0,1,1,0,0,24.0,109.0,9.0,25.0,549.0,44.0
2,0,1,0,1,1,0,0,58.0,43.0,3576.0,0.0,6715.0,49.0
3,0,1,0,1,1,0,0,33.0,0.0,1283.0,371.0,3329.0,193.0
4,1,0,0,1,1,0,0,16.0,303.0,70.0,151.0,565.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,0,1,0,1,0,1,0,41.0,0.0,6819.0,0.0,1643.0,74.0
8689,1,0,0,1,0,0,1,18.0,0.0,0.0,0.0,0.0,0.0
8690,1,0,0,1,1,0,0,26.0,0.0,0.0,1872.0,1.0,0.0
8691,0,1,0,1,0,1,0,32.0,0.0,1049.0,0.0,353.0,3235.0


In [ ]:
df_train[['Transported']].isnull().sum()

Transported    0
dtype: int64

In [ ]:
df_target=df_train[['Transported']]

 # Lets start to use a model to make predictions using gridSearchCV

# Splitting test and train  data:

In [ ]:
X_tr, X_ts, y_tr, y_ts= train_test_split( df_model, df_target, test_size=0.30, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf = RandomForestClassifier(random_state=0)  

In [ ]:
# Parameters that I want to teste
parameters = { 
              'max_features':['auto','sqrt','log2'],
              'n_estimators':[100,200]}

In [ ]:
# Creating the object of GridSearchCV.
grid = GridSearchCV(estimator = rf,          # the model
                    param_grid = parameters,  # dictionary of values
                    scoring = 'f1',           # scoring
                    cv = 20)                  

In [ ]:
#Após a criação do objeto do GridSearchCV, pode-se treinar e imprimir os resultados.

# Treinando o grid.
grid_fitted=grid.fit(X_tr, y_tr)

# Imprimindo os resultados.
#pd.DataFrame(grid_fitted.cv_results_)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 

In [ ]:
# Imprime colunas de interesse.
pd.DataFrame(grid_fitted.cv_results_)[['params','rank_test_score','mean_test_score']]

,params,rank_test_score,mean_test_score
0,"{'max_features': 'auto', 'n_estimators': 100}",4,0.774979
1,"{'max_features': 'auto', 'n_estimators': 200}",1,0.776394
2,"{'max_features': 'sqrt', 'n_estimators': 100}",4,0.774979
3,"{'max_features': 'sqrt', 'n_estimators': 200}",1,0.776394
4,"{'max_features': 'log2', 'n_estimators': 100}",4,0.774979
5,"{'max_features': 'log2', 'n_estimators': 200}",1,0.776394


In [ ]:
# Imprime os parâmetros que produziram o ".best_score_".
## Grid_fitted is the model
grid_fitted.best_params_

{'max_features': 'auto', 'n_estimators': 200}

In [ ]:
y_pred=grid_fitted.predict(X_ts)

In [ ]:
from sklearn import metrics

print('precision: ',metrics.precision_score (y_ts, y_pred))
print('roc_auc: ',metrics.roc_auc_score (y_ts, y_pred))
print('recall: ',metrics.recall_score (y_ts, y_pred))


precision:  0.7717717717717718
roc_auc:  0.7717682895627608
recall:  0.7793783169067475


####  Our model has similar values to precision and recall. That probably means that  fp=fn. This means that our algorithm has classified an equal amount of users as false positives, as it classified false negatives.

 # Now, finally using the df_test data

In [ ]:
df_test

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,9266_02,Earth,True,G/1496/S,TRAPPIST-1e,34.0,False,0.0,0.0,0.0,0.0,0.0,Jeron Peter
4273,9269_01,Earth,False,NaN,TRAPPIST-1e,42.0,False,0.0,847.0,17.0,10.0,144.0,Matty Scheron
4274,9271_01,Mars,True,D/296/P,55 Cancri e,NaN,False,0.0,0.0,0.0,0.0,0.0,Jayrin Pore
4275,9273_01,Europa,False,D/297/P,NaN,NaN,False,0.0,2680.0,0.0,0.0,523.0,Kitakan Conale


In [ ]:
df_test.describe()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
count,4186.000000,4195.000000,4171.000000,4179.000000,4176.000000,4197.000000
mean,28.658146,219.266269,439.484296,177.295525,303.052443,310.710031
std,14.179072,607.011289,1527.663045,560.821123,1117.186015,1246.994742
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,26.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,37.000000,53.000000,78.000000,33.000000,50.000000,36.000000
max,79.000000,11567.000000,25273.000000,8292.000000,19844.000000,22272.000000


# Using the same tratament that we gave to df_train dataframe

In [ ]:
df_test.Age.fillna(28,inplace=True)
df_test.RoomService.fillna(219,inplace=True)
df_test.FoodCourt.fillna(439,inplace=True)
df_test.ShoppingMall.fillna(177,inplace=True)
df_test.Spa.fillna(303,inplace=True)
df_test.VRDeck.fillna(310,inplace=True)

In [ ]:
# Get one hot encoding of columns B
one_hot = pd.get_dummies(df_test['HomePlanet'])

# Join the encoded df
df_test = df_test.join(one_hot)


##----------------------------------
one_hot = pd.get_dummies(df_test['Destination'])
df_test = df_test.join(one_hot)


In [ ]:
df_test['new_CryoSleep']=df_test['CryoSleep'].apply(lambda x:  0 if x== 'False' else 1)

In [ ]:
df_test.columns

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age',
       'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Name', 'Earth', 'Europa', 'Mars', '55 Cancri e', 'PSO J318.5-22',
       'TRAPPIST-1e', 'new_CryoSleep'],
      dtype='object')

In [ ]:
df_test.rename(columns={'55 Cancri e':'55_Cancri_e',
                         'PSO J318.5-22':'PSO_J318_5_22','TRAPPIST-1e':'TRAPPIST_1e'},inplace=True)

In [ ]:
df_model_2 = df_test[['Earth','Europa','Mars','new_CryoSleep',
                     'TRAPPIST_1e','55_Cancri_e','PSO_J318_5_22',
                     'Age','RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']]

# Once we have the model, let's predict what will happen to the people in the df_test dataframe

In [ ]:
ynew = grid_fitted.predict(df_model_2)

In [ ]:
df_test["Transported"] = ynew 

In [ ]:
df_final= df_test[['PassengerId','Transported']]

In [ ]:
df

,PassengerId,Transported
0,0013_01,False
1,0018_01,False
2,0019_01,False
3,0021_01,False
4,0023_01,False
...,...,...
4272,9266_02,False
4273,9269_01,False
4274,9271_01,False
4275,9273_01,False


In [ ]:
df_final

,PassengerId,Transported
0,0013_01,False
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,False
...,...,...
4272,9266_02,True
4273,9269_01,False
4274,9271_01,True
4275,9273_01,True


In [ ]:
# df_final.to_excel("df.xlsx") 

In [ ]:

df_final.to_csv("df.csv", index=False)

In [ ]:
df_final

,PassengerId,predict
0,0013_01,False
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,False
...,...,...
4272,9266_02,True
4273,9269_01,False
4274,9271_01,True
4275,9273_01,True


In [ ]:
df_final

,PassengerId,predict
0,0013_01,False
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,False
...,...,...
4272,9266_02,True
4273,9269_01,False
4274,9271_01,True
4275,9273_01,True
